In [2]:
import tensorflow as tf
from tensorflow.python.ops import variable_scope

In [3]:
tf.enable_eager_execution()

In [4]:
csv = ["21|90,24,46|;31,24|42|56;63||46", "54|84;98|;78|83,36,89", "12,58,12;;83,24"]
slot_name = ["ad_clk-slot_1", "ad_clk-slot_2", "ad_clk-slot_3"]
batch_size, seq_length, slot_num = 3, 4, 3
sample = tf.constant(csv)
print(sample)

tf.Tensor(
[b'21|90,24,46|;31,24|42|56;63||46' b'54|84;98|;78|83,36,89'
 b'12,58,12;;83,24'], shape=(3,), dtype=string)


In [7]:
batch_size, seq_length, slot_num = 3, 4, 3
slot_feas = tf.decode_csv(sample, [[""]]*slot_num, ";")
print(slot_feas)
slot_feas = tf.decode_csv(sample, [""]*slot_num, ";")
print(slot_feas)

[<tf.Tensor: id=24, shape=(3,), dtype=string, numpy=array([b'21|90,24,46|', b'54|84', b'12,58,12'], dtype=object)>, <tf.Tensor: id=25, shape=(3,), dtype=string, numpy=array([b'31,24|42|56', b'98|', b''], dtype=object)>, <tf.Tensor: id=26, shape=(3,), dtype=string, numpy=array([b'63||46', b'78|83,36,89', b'83,24'], dtype=object)>]
[<tf.Tensor: id=33, shape=(3,), dtype=string, numpy=array([b'21|90,24,46|', b'54|84', b'12,58,12'], dtype=object)>, <tf.Tensor: id=34, shape=(3,), dtype=string, numpy=array([b'31,24|42|56', b'98|', b''], dtype=object)>, <tf.Tensor: id=35, shape=(3,), dtype=string, numpy=array([b'63||46', b'78|83,36,89', b'83,24'], dtype=object)>]


In [ ]:
batch_size, seq_length, slot_num = 3, 4, 3
slot_feas = tf.decode_csv(sample, [""]*slot_num, ";")
print(slot_feas)
outer_sparse_feas = {}
for name, slot_seq in zip(slot_name, slot_feas):
    slot_fea = tf.string_split(slot_seq, "|", False)
    total_string = tf.reshape(tf.sparse_to_dense(slot_fea.indices, [batch_size, seq_length], slot_fea.values, ""), [batch_size*seq_length])
    print(total_string)
    sparse_string = tf.string_split(total_string, ",", False)
    print(sparse_string)
    sparse_value = tf.string_to_number(sparse_string.values, out_type=tf.int64)
    outer_sparse_feas[name] = tf.SparseTensor(sparse_string.indices, sparse_value, sparse_string.dense_shape)
print(outer_sparse_feas)

In [50]:
x=tf.constant("1")
v2=tf.get_variable("v1", [2], tf.int32)
with tf.variable_scope("test"):
    v1=tf.get_variable("v1", [2], tf.int32)
    tf.get_variable("v2", [2], tf.int32)
    v3=tf.get_variable("v1", [2], tf.int32)
#print(x.name)
#print(tf.get_default_graph().get_tensor_by_name(v2.name))
print(v2,v1,v3)

<tf.Variable 'v1:0' shape=(2,) dtype=int32, numpy=array([0, 0], dtype=int32)> <tf.Variable 'test/v1:0' shape=(2,) dtype=int32, numpy=array([0, 0], dtype=int32)> <tf.Variable 'test/v1:0' shape=(2,) dtype=int32, numpy=array([0, 0], dtype=int32)>


In [103]:
colum = tf.feature_column.categorical_column_with_identity("ad_clk_slot_1", 100, 0)
feature_cloumn = tf.feature_column.embedding_column(colum, 10)
colum2 = tf.feature_column.categorical_column_with_identity("ad_clk_slot_2", 100, 0)
feature_cloumn2 = tf.feature_column.embedding_column(colum2, 10)
indicator_column = tf.feature_column.indicator_column(colum)

In [104]:
print(outer_sparse_feas["ad_clk-slot_1"])
tf.feature_column.input_layer({"ad_clk_slot_1":outer_sparse_feas["ad_clk-slot_1"], "ad_clk_slot_2":outer_sparse_feas["ad_clk-slot_2"]}, [feature_cloumn, feature_cloumn2])
#tf.feature_column.input_layer({"ad_clk_slot_1":outer_sparse_feas["ad_clk-slot_1"]}, [indicator_column])[1]

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 0]
 [1 1]
 [1 2]
 [4 0]
 [5 0]
 [8 0]
 [8 1]
 [8 2]], shape=(9, 2), dtype=int64), values=tf.Tensor([21 90 24 46 54 84 12 58 12], shape=(9,), dtype=int64), dense_shape=tf.Tensor([12  3], shape=(2,), dtype=int64))


<tf.Tensor: id=3198, shape=(12, 20), dtype=float32, numpy=
array([[-0.04545068,  0.09263412, -0.21177691, -0.28413165, -0.09922945,
         0.08395215,  0.0576439 , -0.17468515,  0.16587205,  0.02918471,
         0.03127137,  0.14813533,  0.02666359, -0.19582944, -0.29743418,
        -0.13055468,  0.11694263, -0.05274741,  0.06934261,  0.15968838],
       [ 0.20044976,  0.23493136, -0.08263146,  0.19873142,  0.04860223,
         0.03680925, -0.14367378,  0.09496699,  0.00830158, -0.07238144,
         0.48050833, -0.17279343, -0.00837831,  0.25290513,  0.06340476,
        -0.18634628, -0.04697398,  0.23448141, -0.12693545,  0.48568743],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.11217245, -0.48398498, -0.04098111,  0.31399626,  0.27900553,
         0.52005559,  0.48375809, -0.11371475, -0.17020144,  0.00897419],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 0]
 [1 1]
 [1 2]
 [4 0]
 [5 0]
 [8 0]
 [8 1]
 [8 2]], shape=(9, 2), dtype=int64), values=tf.Tensor([321  90  24  46  54  84 412 548  12], shape=(9,), dtype=int64), dense_shape=tf.Tensor([12  3], shape=(2,), dtype=int64))


<tf.Tensor: id=1905, shape=(12, 10), dtype=float32, numpy=
array([[ 0.52254075, -0.02677039, -0.08395325,  0.06128124, -0.31251341,
         0.42425579,  0.0705861 , -0.05156358,  0.3776598 ,  0.31427813],
       [ 0.10741454, -0.27143222, -0.16021368, -0.03774109,  0.04317988,
         0.04162496, -0.02815527,  0.0819087 ,  0.11332647, -0.16410284],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.31245029,  0.01536097,  0.173768  ,  0.01731536, -0.14887927,
        -0.10974558,  0.47252876, -0.02163495, -0.35377961,  0.01994451],
       [ 0.25287259,  0.31501347, -0.08505826,  0.35081163, -0.22504589,
         0.45057985,  0.37342882, -0.19483744,  0.10163494,  0.16682829],
       [ 0.        ,  0.        ,  0.        ,  0.        ,

In [96]:
import collections
class test(collections.namedtuple('tuple', ('x', 'y', 'z'))):
    @property
    def name(self):
        return self.x[0]
    
    def parse(self, inputs):
        self.y = inputs
    
    def fn(self, )

out = test([1,2], '13', 43)
print(out.name)
print(out.z)

1
43
